# Demostrate the DP algorithm on the shortest path problem of an acyclic network
* Suppose T is a network with directed arc. We start from origin $s$, and want to go to the destination $t$.
* We want to find the best route, we can define, on each node $i$ that
    * $f(i)$ is the lenght of the shortest path from $i$ to $t$
* Then, the DP formulation says: $f(i) = \min_j \{f(i) + d_{ij}\}$, where $d_{ij}$ is the length of the arc from $i$ to $j$
* If the network is acyclic, we can always re-label the nodes so that, an directed arc $(i, j)$ belongs to the network implies $i<j$. The next figure is such an example
<img src ='../fig/ShortestPathDemoEg2.png' alt ='acylic-network' width="500">
* With such a re-labelling, one way to solve the problem is to work backward from the destination.
    * $f(t)=0$ by definition
    * Walking backward from $i=t-1$ to $i=s\equiv 1$: 
        * $f(i) = \min_{j>i} \{f(i) + d_{ij}\}$
        * let $u(i)$ be the next stop from $i$ to $t$ on the shortest route. Then $u(i) = \arg\min_{j>i} \{f(i) + d_{ij}\}$
* Using the above figure's example, we will write demonstrate the solution
* We will use two different reprentation of the network: dictionary and np.ndarray (like a matrix)

## Using a dictionary representation of the graph

In [1]:
# the data of the graph
nodes = {i for i in range(0,9+1)}
T= dict()
T[0]={1:2,2:5}
T[1]={2:1,3:2}
T[2]={4:6,5:12}
T[3]={4:3,6:4}
T[4]={5:4, 6:3, 7:15, 8:7}
T[5]={7:7}
T[6]={8:7,9:15}
T[7]={9:3}
T[8]={9:10}
T[9]={9:0} #destination, can be skipped

In [2]:
def SPP(s, t, T):    
    '''Solving the Shortest path from s to t
    Input:
        s: label of the starting point of T
        t: label of the destination of T
        T: the dict representation of network with directed arcs
            T[i] itself if a dictionary representing the j:d_ij of arc from i to j
    Assume: 1. for any arc (i,j), i<j; 
            2. t is the end point. s can be any point desired
    Return a dict such that sol[i]=(best_next_point_towards_j, len_best_route)
    '''
    sol = {k:(0,0) for k in T} #initialize solution
    sol[t] = (t,0) #from t the next step t with length zero
    for i in range(t-1, s-1,-1):
        best_val, best_j = float('inf'), None
        for j, dij in T[i].items():
            tmp = sol[j][1] + dij
            if best_val > tmp:
                best_val = tmp
                best_j = j
        sol[i] = (best_j,best_val)
    return sol

In [3]:
sol = SPP(s=0,t=9, T=T)

In [4]:
sol

{0: (1, 21),
 1: (3, 19),
 2: (4, 20),
 3: (4, 17),
 4: (5, 14),
 5: (7, 10),
 6: (9, 15),
 7: (9, 3),
 8: (9, 10),
 9: (9, 0)}

In [5]:
def length_route(route,T):
    '''Given a route as an ordered sequence from some node to another
    Return the length of the route
    T is the dict representation of the graph
    '''
    length = 0
    for k in range(len(route)-1):
        i,j=route[k], route[k+1]
        try:
            length += T[i][j]
        except: #T[i][j] not exists, the route is invalid
            print(f'route {i}->{j} is not valid in T')
            return float('inf') #length = inf
    return length

In [6]:
def verify_sol(sol,s,t,T):
    '''Given a solution sol, verify whether
    the route from s to t indicated by sol has the same len as sol[s][1]
    return True if yes, otherwise return False
    '''
    ZERO = 1e-10 #numerical zero
    route = [s]
    i = s
    while i!=t:
        j = sol[i][0]
        route.append(j)
        i = j #move to j
    return abs(sol[s][1]-length_route(route,T))<ZERO

In [7]:
length_route((0,1,3,4,5,7,9),T)

21

In [8]:
for i in nodes:
    print(f'for node {i}, Is length of path in sol valid? {verify_sol(sol,i,9,T):}')

for node 0, Is length of path in sol valid? True
for node 1, Is length of path in sol valid? True
for node 2, Is length of path in sol valid? True
for node 3, Is length of path in sol valid? True
for node 4, Is length of path in sol valid? True
for node 5, Is length of path in sol valid? True
for node 6, Is length of path in sol valid? True
for node 7, Is length of path in sol valid? True
for node 8, Is length of path in sol valid? True
for node 9, Is length of path in sol valid? True


## Using a Matrix representation
* T[i][j] is the length of the arc (i,j), except that i and j are now python indices (starting from 0)

In [9]:
import numpy as np

In [10]:
def n2p(i): return i-1 #natural index to python index
def p2n(i): return i+1 #convert python index to the natural one

In [11]:
s, t = 0, 9
nodes={i for i in range(s,t+1)}
N = len(nodes)
T = np.ones((N,N))*np.inf

<img src = "../fig/ShortestPathDemoEg2.png" alt = "shortest path demo" width = "500">

In [12]:
T[0,1] = 2; T[0,2] = 5
T[1,2] = 1; T[1,3] = 2; 
T[2,4] = 6; T[2,5] = 12;
T[3,4] = 3; T[3,6] = 4;
T[4,5] = 4; T[4,6] = 3; T[4,7] = 15; T[4,8] = 7;
T[5,7] = 7;
T[6,8] = 7; T[6,9] = 15;
T[7,9] = 3;
T[8,9] = 10;

In [13]:
def SSP_M(T):
    '''
    Solving the Shortest path from s to t 
    Input:
        T: the matrix representation of network with directed arcs
            T[i,j] is the length of arc (i,j)
    Assume: 1. for any arc (i,j), i<j; 
            2. 0 is the starting point. np.shape(T)[0]-1 is the destination
    Return two arrays:  
        u[i]= from i to tbest point to move to 
        f[i]= length of best route from i to t
    '''
    N = np.shape(T)[0]
    u, f = np.zeros(N,int), np.ones(N)*np.inf
    u[-1], f[-1] = N-1, 0 # from destination to destination, no distance
    for i in range(N-1-1,0-1, -1):
        # since f[i] = \min_{j>i} { f[j] + T[i,j] }
        u[i] = np.argmin(f[i+1:] + T[i,i+1:]) + i+1
        f[i] = f[u[i]] + T[i,u[i]]
    return u, f

In [14]:
u, f = SSP_M(T)

In [15]:
N = np.shape(T)[0]
sol = {i:(u[i],f[i]) for i in range(N)}
sol

{0: (1, 21.0),
 1: (3, 19.0),
 2: (4, 20.0),
 3: (4, 17.0),
 4: (5, 14.0),
 5: (7, 10.0),
 6: (9, 15.0),
 7: (9, 3.0),
 8: (9, 10.0),
 9: (9, 0.0)}